In [1]:
%load_ext autoreload
%autoreload 2

import sys
# can't relative import with sister directory
sys.path.insert(0, "..")
import ipywidgets
from ai2es import gui
import pandas as pd

In [2]:
# USER CHANGES
year = 2017
time_of_day = 'night' # or 'day'
precip = 'no_precip'
precip_threshold = 0.1  # threshold of rain gauge output be to be considered precipitating
labels = ["precip", "no_precip", "obstructed"]

In [3]:
df = pd.read_parquet(f'/ai2es/matched_parquet/{year}_timeofday.parquet')
df = df[df['night'] == True if time_of_day == 'night' else df['night'] == False]
df = df[df['precip_accum_1min [mm]'] > precip_threshold if precip == 'precip' else df['precip_accum_1min [mm]'] == 0.0]
folder_dest = f'/ai2es/{time_of_day}_{precip}_hand_labeled/{year}'

# make folders to save to if they don't exist
for label in labels:
    save_path = os.path.join(folder_dest, label)
    if not os.path.exists(save_path):
        os.makedirs(save_path)

In [4]:
g = gui.GUI(df['path'], labels, folder_dest)
g.index = 8700  # pick up at a specific index if restarting
g.display_image()
g.make_buttons()
image_box = ipywidgets.HBox([g.center])
label_btns = ipywidgets.HBox([g.buttons[0], g.buttons[1], g.buttons[2]])
undo = ipywidgets.HBox([g.undo_btn])
box_layout = ipywidgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')
display(ipywidgets.VBox([image_box, label_btns, undo], layout=box_layout))